#### This is a trial of Dask Client Usage. We have 32 GB DataSet but 16 GB Ram. Therefore we can use Paralellism Concept with Dask not to overload Ram.

In [1]:
import pandas as pd
from dask import dataframe as dd
from dask.distributed import Client, progress

In [19]:
from datetime import datetime

#### A Dask DataFrame is a large parallel DataFrame composed of many smaller Pandas DataFrames, split along the index. These Pandas DataFrames may live on disk for larger-than-memory computing on a single machine, or on many different machines in a cluster. One Dask DataFrame operation triggers many operations on the constituent Pandas DataFrames.

In [2]:
c = Client()
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.75 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57692,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.75 GiB
Comm: tcp://127.0.0.1:57718,Total threads: 2
Dashboard: http://127.0.0.1:57720/status,Memory: 3.94 GiB
Nanny: tcp://127.0.0.1:57697,


In [10]:
dfd_14 = dd.read_csv('infina-export-2021-11-08-batch_*.csv')

In [36]:
dfd_14.columns

Index(['idx', 'id', 'currency', 'mref', 'transactionCode',
       'payment_partner_name', 'payment_partner_creditorId',
       'payment_partner_Iban', 'payment_partner_bic', 'bookingDate', 'purpose',
       'tags', 'amount', 'userId'],
      dtype='object')

In [37]:
dfd_14.shape

(Delayed('int-c09bbe29-c795-4592-b65e-17cb59ed1981'), 14)

In [6]:
# dfd_3

In [7]:
# %time dfd_3.groupby("userId")["amount"].sum().compute().sort_values()

Wall time: 9min 51s


userId
bankloginid.606ed645ceb33961b7e8f0bb   -54268055.46
60d1f26901ffc90396a4249a               -20820257.21
bankloginid.5fd0ede9e4f2a945b4f4a857   -19461712.24
af3217c0-49e5-4d28-a42b-01e54f3be6c0   -18337451.62
3c9904ac-9b0e-4e42-9615-682e814108ea   -16122464.17
                                           ...     
bankloginid.5ea1a232bfe80500011a7ba2    31328906.28
bankloginid.5e996f75745fae00011ab79c    31406280.05
bankloginid.5e96cc08e6585700014802b1    31470398.58
bankloginid.5e9809ffbfe80500011a4a70    31525547.27
bankloginid.5e96bf76e658570001480230    31542084.03
Name: amount, Length: 75817, dtype: float64

In [46]:
%time dfd_5 = dfd_14[['bookingDate', 'purpose','tags', 'amount', 'userId']]

Wall time: 42.5 ms


In [47]:
dfd_5.head()

,bookingDate,purpose,tags,amount,userId
0,1534294861,Kartenzahlung <<FIRST_NAME=ab25627f0022499b0d3...,"card_payment,spending,mobility,car,car_fuel,fi...",-47.99,bankloginid.60502bc3c395af21ec04b3e8
1,1607040000,STAR TST/BREMEN/DE 03.12.2020 um 21:04:18 Uhr ...,"card_payment,spending,mobility,car,car_fuel,fi...",-49.37,60267b72afc354135d9bbd73
2,1571184000,2019-10-15T19:51 Debitk.0 2021-12 KARTENZAHLUNG,"card_payment,spending,variabel",-17.15,6094f2f2fcac366879c1a810
3,1557705600,2019-05-11T14:39 Debitk.2 2022-12 KARTENZAHLUNG,"card_payment,spending,variabel",-106.96,60bf47a86b6759357385e6d9
4,1622592000,KARTENZAHLUNG 2021-06-01T15:45 Debitk.2 2024-1...,"card_payment,spending,household,household_fash...",-41.96,60be2816b92620310a5e5ca0


In [34]:
dfd_5.shape

(Delayed('int-426d9dee-0c00-4381-b07b-8829c5209245'), 5)

### How many users are there?

In [18]:
dfd_5['userId'].nunique().compute()  # 4m30s

75817

In [18]:
# %time dfd_3.groupby("userId")["amount"].sum().compute().sort_values()

Wall time: 7min 37s


userId
bankloginid.606ed645ceb33961b7e8f0bb   -54268055.46
60d1f26901ffc90396a4249a               -20820257.21
bankloginid.5fd0ede9e4f2a945b4f4a857   -19461712.24
af3217c0-49e5-4d28-a42b-01e54f3be6c0   -18337451.62
3c9904ac-9b0e-4e42-9615-682e814108ea   -16122464.17
                                           ...     
bankloginid.5ea1a232bfe80500011a7ba2    31328906.28
bankloginid.5e996f75745fae00011ab79c    31406280.05
bankloginid.5e96cc08e6585700014802b1    31470398.58
bankloginid.5e9809ffbfe80500011a4a70    31525547.27
bankloginid.5e96bf76e658570001480230    31542084.03
Name: amount, Length: 75817, dtype: float64

### Sorting Data By UserId

In [42]:
sorted_dfd = dfd_5.sort_values(by="userId", npartitions=None, ascending=True, na_position='last')

In [43]:
sorted_dfd.head()

,bookingDate,purpose,tags,amount,userId
293,1621900800,2021-05-20T12:58 Debitk.2 2099-12,"card_payment,spending,fix-variabel",-117.47,0012c035-d1c8-4dd0-a175-4785c00cdb61
5736,1619481600,2021-04-26T13:10 Debitk.2 2024-12,"card_payment,spending,health,pharmacy,fix-vari...",-6.97,0012c035-d1c8-4dd0-a175-4785c00cdb61
7332,1612137600,RATE PER 01.02.2021,"direct_debit,spending,financing,credit_rate,fi...",-141.40,0012c035-d1c8-4dd0-a175-4785c00cdb61
85,1609286400,Entgeltabrechnung siehe Anlage\nAbrechnung 30....,"invoice,spending,bank_charges,account_manageme...",-0.08,0012c035-d1c8-4dd0-a175-4785c00cdb61
4927,1614902400,2021-03-03T18:24 Debitk.1 2022-12,"card_payment,spending,health,pharmacy,variabel",-35.14,0012c035-d1c8-4dd0-a175-4785c00cdb61


In [44]:
sorted_dfd_small = sorted_dfd.loc[:100000]

In [48]:
sorted_dfd_small.shape

(Delayed('int-5c233be9-5c96-49a0-b9a7-9328e8dd27b1'), 5)

### Converting UnixTimeStamp into Readable Format 

In [28]:
print(
    datetime.fromtimestamp(
        int("1534294861")
    ).strftime('%Y-%m-%d-%H-%')
)

2018-08-15


In [39]:
dfd_5.head()

,bookingDate,purpose,tags,amount,userId
0,1534294861,Kartenzahlung <<FIRST_NAME=ab25627f0022499b0d3...,"card_payment,spending,mobility,car,car_fuel,fi...",-47.99,bankloginid.60502bc3c395af21ec04b3e8
1,1607040000,STAR TST/BREMEN/DE 03.12.2020 um 21:04:18 Uhr ...,"card_payment,spending,mobility,car,car_fuel,fi...",-49.37,60267b72afc354135d9bbd73
2,1571184000,2019-10-15T19:51 Debitk.0 2021-12 KARTENZAHLUNG,"card_payment,spending,variabel",-17.15,6094f2f2fcac366879c1a810
3,1557705600,2019-05-11T14:39 Debitk.2 2022-12 KARTENZAHLUNG,"card_payment,spending,variabel",-106.96,60bf47a86b6759357385e6d9
4,1622592000,KARTENZAHLUNG 2021-06-01T15:45 Debitk.2 2024-1...,"card_payment,spending,household,household_fash...",-41.96,60be2816b92620310a5e5ca0


In [1]:
# dfd_5["date"]=pd.to_datetime(dfd_5.bookingDate, unit="D")

In [2]:
# dfd_5["bookingDate"] = dfd_5["bookingDate"].apply(lambda d: datetime.fromtimestamp(int(d)).strftime("%Y-%m-%d"))

In [3]:
# dfd_5[dfd_5["bookingDate"]].head()

In [26]:
dfd_3.isna().sum().compute()

bookingDate         0
purpose        360833
tags           335553
amount              0
userId              0
dtype: int64

In [27]:
len(dfd_3.index)

66381917

In [28]:
dfd_4 = dfd_3.dropna()

In [29]:
len(dfd_4.index)

65691541

In [30]:
%time dfd_4.isna().sum().compute()

Wall time: 6min 25s


bookingDate    0
purpose        0
tags           0
amount         0
userId         0
dtype: int64

In [14]:
dfd = dd.read_csv('combined-csv-files.csv', dtype={'amount': 'object',
       'bookingDate': 'object',
       'idx': 'object',
       'transactionCode': 'object'}, engine='python', encoding='utf-8')

In [55]:
dfd_2 = dd.read_csv('infina-export-2021-11-08-batch_0.csv', dtype={'amount': 'object',
       'bookingDate': 'object',
       'idx': 'object',
       'transactionCode': 'object'}, engine='python', encoding='utf-8')

In [32]:
dfd

,idx,id,currency,mref,transactionCode,payment_partner_name,payment_partner_creditorId,payment_partner_Iban,payment_partner_bic,bookingDate,purpose,tags,amount,userId
npartitions=507,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [33]:
# dfd = dfd.persist()
# dfd

In [34]:
"""dfd = dd.read_csv(
    'combined-csv-files.csv', 
    sep=',',
    names=['idx', 'id', 'currency', 'mref', 'transactionCode',
       'payment_partner_name', 'payment_partner_creditorId',
       'payment_partner_Iban', 'payment_partner_bic', 'bookingDate', 'purpose',
       'tags', 'amount', 'userId'],
    # dtype={
    #    ‘referrer_type’: ‘category’, 
    #    ’n’: ‘uint32’},
    blocksize=64000000, # = 64 Mb chunks
    engine='python',
    quotechar='"',
    quoting=3
)"""

'dfd = dd.read_csv(\n    \'combined-csv-files.csv\', \n    sep=\',\',\n    names=[\'idx\', \'id\', \'currency\', \'mref\', \'transactionCode\',\n       \'payment_partner_name\', \'payment_partner_creditorId\',\n       \'payment_partner_Iban\', \'payment_partner_bic\', \'bookingDate\', \'purpose\',\n       \'tags\', \'amount\', \'userId\'],\n    # dtype={\n    #    ‘referrer_type’: ‘category’, \n    #    ’n’: ‘uint32’},\n    blocksize=64000000, # = 64 Mb chunks\n    engine=\'python\',\n    quotechar=\'"\',\n    quoting=3\n)'

In [10]:
dfd.dtypes

idx                            object
id                             object
currency                       object
mref                           object
transactionCode                object
payment_partner_name           object
payment_partner_creditorId     object
payment_partner_Iban           object
payment_partner_bic            object
bookingDate                    object
purpose                        object
tags                           object
amount                        float64
userId                         object
dtype: object

In [36]:
dfd.shape

(Delayed('int-69036acd-ce9c-4d6a-85f3-ceb02ab2777d'), 14)

In [37]:
dfd.columns

Index(['idx', 'id', 'currency', 'mref', 'transactionCode',
       'payment_partner_name', 'payment_partner_creditorId',
       'payment_partner_Iban', 'payment_partner_bic', 'bookingDate', 'purpose',
       'tags', 'amount', 'userId'],
      dtype='object')

In [56]:
dfd_2.columns

Index(['idx', 'id', 'currency', 'mref', 'transactionCode',
       'payment_partner_name', 'payment_partner_creditorId',
       'payment_partner_Iban', 'payment_partner_bic', 'bookingDate', 'purpose',
       'tags', 'amount', 'userId'],
      dtype='object')

In [52]:
%time dfd.groupby("userId")["amount"].sum()

Wall time: 13.6 ms


Dask Series Structure:
npartitions=1
    object
       ...
Name: amount, dtype: object
Dask Name: series-groupby-sum-agg, 1087 tasks

In [12]:
%time dfd.loc[0:1000]

Wall time: 7.85 ms


,idx,id,currency,mref,transactionCode,payment_partner_name,payment_partner_creditorId,payment_partner_Iban,payment_partner_bic,bookingDate,purpose,tags,amount,userId
npartitions=507,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
%time dfd.loc[0:1000].compute()

In [57]:
users = dfd.loc[dfd["userId"].isin(["bankloginid"]),["userId","tags", "amount", "bookingDate"]].groupby(["userId"])

In [58]:
store = pd.HDFStore('clickstream_store2.h5')

In [59]:
top_users = users.sum().nlargest(10, "amount")

In [60]:
top_users

""
npartitions=1
""
""


In [ ]:
# Read the csv file
data = pd.read_csv(
    'combined-csv-files.csv',
    delimiter=',',
    names=['idx', 'id', 'currency', 'mref', 'transactionCode',
       'payment_partner_name', 'payment_partner_creditorId',
       'payment_partner_Iban', 'payment_partner_bic', 'bookingDate', 'purpose',
       'tags', 'amount', 'userId'],
    )

In [27]:
data.shape

(66358824, 14)

In [8]:
data = pd.read_csv('infina-export-2021-11-08-batch_0.csv')

In [31]:
# data = pd.read_csv('infina-export-2021-11-08-batch_1.csv')

In [9]:
data.head()

,idx,id,currency,mref,transactionCode,payment_partner_name,payment_partner_creditorId,payment_partner_Iban,payment_partner_bic,bookingDate,purpose,tags,amount,userId
0,1,9f158a2d630637e3b73ac4200039fc0785f12205ecd939...,EUR,NaN,999.0,<<PP_NAME=ab25627f0022499b0d37259f87b791bba77a...,NaN,<<PP_IBAN=d23e8edce1c3b7fe4deace094ab9c0bb1b81...,NaN,1534294861,Kartenzahlung <<FIRST_NAME=ab25627f0022499b0d3...,"card_payment,spending,mobility,car,car_fuel,fi...",-47.99,bankloginid.60502bc3c395af21ec04b3e8
1,2,754fa65ff2bccbf0bcf087ad074e8d7f2acd1b8397b8c2...,EUR,<<MREF=7ef58601872cd2567ea2879a38cd27fcea08908...,106.0,Orlen Deutschland GmbH,NaN,DE39500500000096500038,HELADEFFXXX,1607040000,STAR TST/BREMEN/DE 03.12.2020 um 21:04:18 Uhr ...,"card_payment,spending,mobility,car,car_fuel,fi...",-49.37,60267b72afc354135d9bbd73
2,3,7dd6cbe264f4d6c1ff7de82030d9002c96361938c99625...,EUR,NaN,106.0,<<PP_NAME=9f231e1f69fb954bc40ee46465ef75458dd8...,NaN,<<PP_IBAN=00ab73d6ce23cd8026554cb6b41c72b636ee...,WELADEDDXXX,1571184000,2019-10-15T19:51 Debitk.0 2021-12 KARTENZAHLUNG,"card_payment,spending,variabel",-17.15,6094f2f2fcac366879c1a810
3,4,cd06616ca8dfa487c0eeaded1dd41fabdd9284680830f4...,EUR,NaN,106.0,GARTENC. FULDABRUECK 1,NaN,DE78520400210331190900,COBADEFFXXX,1557705600,2019-05-11T14:39 Debitk.2 2022-12 KARTENZAHLUNG,"card_payment,spending,variabel",-106.96,60bf47a86b6759357385e6d9
4,5,01e8dd98a9ac8dab6bc013886fae42cf11d9e6d0d56159...,EUR,NaN,999.0,<<PP_NAME=362cd0e4912e4d78fb2201f8da6434c6ace3...,NaN,<<PP_IBAN=ac0127b5365b7de870ff07046b2c20f0882a...,COBADEDDXXX,1622592000,KARTENZAHLUNG 2021-06-01T15:45 Debitk.2 2024-1...,"card_payment,spending,household,household_fash...",-41.96,60be2816b92620310a5e5ca0


In [50]:
data.groupby("userId")["amount"].sum().sort_values()

userId
29e0dc95-52da-4274-9679-14d981bf3c84                                   -157865.12
bankloginid.5e627bfd6ff2fc00019da821                                    -61343.08
60abeea522e6ba4e851482bf                                                -50612.11
609908cf13aef1606cac76af                                                -16413.36
618527c14494e74e23a227c5                                                -15297.54
                                                                          ...    
bankloginid.5eb671b4f8fcfd0001eff7b2                                     34571.95
6059e94ca60d8a28cb7a54ab                                                 35869.53
bankloginid.605df8526732b330e386e617                                     44146.60
<<USER_ID=3d4aa868034b94756d7e63ade5c946c6dc5ff93e5bb1e84fb5856a83>>     47101.88
5fc2813660fc9f2580d2ef79                                                 70980.85
Name: amount, Length: 6922, dtype: float64

In [32]:
data.columns

Index(['idx', 'id', 'currency', 'mref', 'transactionCode',
       'payment_partner_name', 'payment_partner_creditorId',
       'payment_partner_Iban', 'payment_partner_bic', 'bookingDate', 'purpose',
       'tags', 'amount', 'userId'],
      dtype='object')

In [33]:
data[data.userId=="bankloginid.60502bc3c395af21ec04b3e8"]

,idx,id,currency,mref,transactionCode,payment_partner_name,payment_partner_creditorId,payment_partner_Iban,payment_partner_bic,bookingDate,purpose,tags,amount,userId
1,1,9f158a2d630637e3b73ac4200039fc0785f12205ecd939...,EUR,NaN,999,<<PP_NAME=ab25627f0022499b0d37259f87b791bba77a...,NaN,<<PP_IBAN=d23e8edce1c3b7fe4deace094ab9c0bb1b81...,NaN,1534294861,Kartenzahlung <<FIRST_NAME=ab25627f0022499b0d3...,"card_payment,spending,mobility,car,car_fuel,fi...",-47.99,bankloginid.60502bc3c395af21ec04b3e8
168795,8128774,1a94e2d0deb30368b33ab7fda7a4fa48f9bbc871e52d2f...,EUR,<<MREF=1dd656a9eb4bf440fe866e0837d5f2a404db667...,999,Telekom Deutschland GmbH,DE93ZZZ00000078611,DE75370800400232113001,NaN,1578358861,Telekom Deutschland GmbH Festnetz Vertragskont...,"direct_debit,SEPA_recurrent,spending,telecommu...",-39.99,bankloginid.60502bc3c395af21ec04b3e8
332835,8276794,9b2b10f9bb9f177be15292ee6de64a1b6746f1eeff7cbd...,EUR,NaN,999,<<PP_NAME=f9a93395fc2086c4fdfeb60f228a87f0e24f...,NaN,<<PP_IBAN=f55ee11d163c5a5c2622b6407306b1bdd2ed...,NaN,1538874061,"Kartenzahlung EDEKA, Musterstadt//Musterstadt/...","card_payment,spending,household,household_groc...",-72.88,bankloginid.60502bc3c395af21ec04b3e8
409914,8345863,85b10c1f64c79a83b1f7c2a743b142f77d411789b551e8...,EUR,<<MREF=81ceb63db1e212c953cee248eb4003099902a43...,999,H+M 000 SAGT VIELEN DANK,DE95ZZZ00001678202,DE62200400000244890003,NaN,1550970061,H+M 000 SAGT VIELEN DANK ELV11<<[0-9]{3}>>111 ...,"card_payment,direct_debit,SEPA_one-off,spendin...",-0.1,bankloginid.60502bc3c395af21ec04b3e8
452621,8388565,546fa5c11b722c7dec2a2fb571c1ddd8b0ad10ca1e83cd...,EUR,NaN,999,"DANKE, IHR LIDL",NaN,DE61300500000008000119,NaN,1560819661,"Kartenzahlung DANKE, IHR LIDL//Musterstadt/DE ...","card_payment,spending,household,household_groc...",-36.41,bankloginid.60502bc3c395af21ec04b3e8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66133826,7791032,6eeaf693cd2d16896c3be250940da06d4d585c52547f3b...,EUR,NaN,999,<<PP_NAME=ab25627f0022499b0d37259f87b791bba77a...,NaN,<<PP_IBAN=d23e8edce1c3b7fe4deace094ab9c0bb1b81...,NaN,1606266061,Kartenzahlung <<FIRST_NAME=ab25627f0022499b0d3...,"card_payment,spending,mobility,car,car_fuel,fi...",-47.99,bankloginid.60502bc3c395af21ec04b3e8
66150586,7807790,8e3ce1038589d611ebfdabba8461d85d540ab903203977...,EUR,<<MREF=81ceb63db1e212c953cee248eb4003099902a43...,999,H+M 000 SAGT VIELEN DANK,DE95ZZZ00001678202,DE62200400000244890003,NaN,1590282061,H+M 000 SAGT VIELEN DANK ELV11<<[0-9]{3}>>111 ...,"card_payment,direct_debit,SEPA_one-off,spendin...",-0.1,bankloginid.60502bc3c395af21ec04b3e8
66240348,7889540,61a46742ee5085a94c46b30d4b01f09d093ac9c45a4ffc...,EUR,<<MREF=da67a9c42c606029f28dae7c381d593a37edbfb...,999,"PayPal (Europe) S.a.r.l. et Cie., S .C.A.",LU96ZZZ0000000000000000058,DE88500700100175526303,NaN,1582765261,"PayPal (Europe) S.a.r.l. et Cie., S .C.A. PP.8...","direct_debit,paypal_debit,paypal,SEPA_recurren...",-9.99,bankloginid.60502bc3c395af21ec04b3e8
66284500,7925687,d9cd7550ce2c80be0f51c8bfab2b5ba28409022b8d8e9b...,EUR,NaN,999,"DANKE, IHR LIDL",NaN,DE61300500000008000119,NaN,1540256461,"Kartenzahlung DANKE, IHR LIDL//Musterstadt/DE ...","card_payment,spending,household,household_groc...",-36.41,bankloginid.60502bc3c395af21ec04b3e8


In [34]:
data[data.userId=="60502bc3c395af21ec04b3e8"]

,idx,id,currency,mref,transactionCode,payment_partner_name,payment_partner_creditorId,payment_partner_Iban,payment_partner_bic,bookingDate,purpose,tags,amount,userId


In [ ]:
data.value_counts()

In [ ]:
data["userId"].value_counts()

In [12]:
data = data[['bookingDate', 'purpose','tags', 'amount', 'userId']]

In [13]:
data

,bookingDate,purpose,tags,amount,userId
0,1534294861,Kartenzahlung <<FIRST_NAME=ab25627f0022499b0d3...,"card_payment,spending,mobility,car,car_fuel,fi...",-47.99,bankloginid.60502bc3c395af21ec04b3e8
1,1607040000,STAR TST/BREMEN/DE 03.12.2020 um 21:04:18 Uhr ...,"card_payment,spending,mobility,car,car_fuel,fi...",-49.37,60267b72afc354135d9bbd73
2,1571184000,2019-10-15T19:51 Debitk.0 2021-12 KARTENZAHLUNG,"card_payment,spending,variabel",-17.15,6094f2f2fcac366879c1a810
3,1557705600,2019-05-11T14:39 Debitk.2 2022-12 KARTENZAHLUNG,"card_payment,spending,variabel",-106.96,60bf47a86b6759357385e6d9
4,1622592000,KARTENZAHLUNG 2021-06-01T15:45 Debitk.2 2024-1...,"card_payment,spending,household,household_fash...",-41.96,60be2816b92620310a5e5ca0
...,...,...,...,...,...
7995,1597017600,Nachzahlung Kutzner GUTSCHR. UEBERWEISUNG,"earning,fix,recurring",191.84,617268782801a40992a19344
7996,1531270861,AUSZUBILDENDEN-GEHALT LAUFENDER MONAT,"earning,salary,salary_apprenticeship,fix,recur...",1182.61,bankloginid.608d333bfb57e64ef03dfc74
7997,1595203200,oilwashed hose 46 GUTSCHR. UEBERWEISUNG,"earning,variabel",14.99,60faa328044fcd15644316e9
7998,1592352000,<<STREET_PREFIX=4cc88562e7ad43ddf45746bfb5f0cb...,"earning,variabel",31.20,60c85b14c58d270fa375b821


In [23]:
data.userId.value_counts()

8d713472-c899-4b4d-9ef2-4038f4b970b5                                    37
<<USER_ID=10e97a34e98819cf5fc317247bcda9439d9f836b9760470d88a1b553>>    23
04e22d4d-7d1b-4282-ae66-da806005f521                                    23
23f4d7b8-9253-4ed5-a4e4-acd4260320d0                                    21
f820d474-2abd-4a15-8d75-0a1b1e013b10                                    14
                                                                        ..
61759fa342e7d1501a5c2884                                                 1
61161cbcef4b29164055938e                                                 1
611b53862dd3c3401102c295                                                 1
611293ea0a8662223d370c7c                                                 1
60b25915da934e1061dcc716                                                 1
Name: userId, Length: 6922, dtype: int64

In [24]:
data.isna().sum()

bookingDate      0
purpose        106
tags            29
amount           0
userId           0
dtype: int64